In [37]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_2"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


#Werktag
df['Werktag'] = (
    df[['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag']].sum(axis=1) > 0
) & (df['Feiertag'] == 0)
# In 0/1 umwandeln
df['Werktag'] = df['Werktag'].astype(int)

# Sicherstellen, dass "Datum" im Datumsformat ist
df['Datum'] = pd.to_datetime(df['Datum'])
# Jahreszeiten zuweisen
df['Frühling'] = df['Datum'].dt.month.isin([3, 4, 5]).astype(int)
df['Sommer'] = df['Datum'].dt.month.isin([6, 7, 8]).astype(int)
df['Herbst'] = df['Datum'].dt.month.isin([9, 10, 11]).astype(int)
df['Winter'] = df['Datum'].dt.month.isin([12, 1, 2]).astype(int)

#Number of Ships skalieren
# Min-Max-Skalierung manuell berechnen
df['Number_of_ships_scaled'] = (
    df['Number_of_ships'] - df['Number_of_ships'].min()
) / (df['Number_of_ships'].max() - df['Number_of_ships'].min())

#Bewölkung zsmfassen
df['Cloud_ok'] = (
    df[['Cloud_Clear', 'Cloud_Partly_Cloudy']].sum(axis=1) > 0)
# In 0/1 umwandeln
df['Cloud_ok'] = df['Cloud_ok'].astype(int)

#wind zsmfassen
df['Wind_ok'] = (
    df[['Wind_Light', 'Wind_Moderate']].sum(axis=1) > 0)
# In 0/1 umwandeln
df['Wind_ok'] = df['Wind_ok'].astype(int)

#neue Wetterkategorien
data_wetter["date"] = pd.to_datetime(data_wetter["date"])
df = df.merge(data_wetter, how="left", left_on="Datum", right_on="date").drop(columns=["date"])

df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
0     535.856285               0          0          1          0         0   
1     546.780787               0          0          1          0         0   
2     427.343259               0          0          0          1         0   
3     454.859641               0          0          1          0         0   
4     492.818804               0          0          1          0         0   
...          ...             ...        ...        ...        ...       ...   
1814  677.415844               0          0          0          0         1   
1815  716.576825               0          0          0          0         1   
1816  721.822447               0          0          0          1         0   
1817  616.358562               0          0          0          0         1   
1818  586.081666               0          0          0          0         1   

      Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloud

In [38]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                    1.000000
Sommer                    0.565432
is_holiday                0.402799
Sonntag                   0.385942
Temp_Warm                 0.319574
Number_of_ships           0.261063
Number_of_ships_scaled    0.261063
Ship                      0.254392
wetter_sehr_schön_x       0.244083
wetter_sehr_schön_y       0.238536
Samstag                   0.224064
Temp_Hot                  0.207087
Cloud_ok                  0.203615
Temp_Mild                 0.185650
Feiertag                  0.170925
KielerWoche               0.155228
Cloud_Clear               0.119683
Cloud_Partly_Cloudy       0.117566
Ostertag                  0.089856
Wind_Moderate             0.085326
Silvester                 0.074378
Heimspiel                 0.061620
Weather_Light_Issues      0.037564
Wind_ok                   0.027271
Markt                     0.017463
Weather_Good              0.010019
Weather_Severe           -0.018105
Wind_Strong              -0.027271
Weather_Moderate    

In [39]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


VPI                       0.263826
Sommer                    0.200458
Werktag                   0.165373
Temp_Cold                 0.125101
Sonntag                   0.103903
is_holiday                0.091818
Winter                    0.083544
Temp_Warm                 0.060112
Temp_Mild                 0.056648
Ship                      0.053259
Number_of_ships_scaled    0.046144
Number_of_ships           0.046038
wetter_sehr_schön_x       0.036581
Cloud_Partly_Cloudy       0.035926
wetter_sehr_schön_y       0.035051
Samstag                   0.034021
Frühling                  0.030156
Cloud_Cloudy              0.029113
Cloud_ok                  0.029113
Herbst                    0.028793
Dienstag                  0.026578
Weihnachtsmarkt           0.023151
KielerWoche               0.020005
Mittwoch                  0.019813
Temp_Hot                  0.018799
Feiertag                  0.018543
Donnerstag                0.016943
wetter_sehr_schlecht_x    0.012164
wetter_sehr_schlecht

In [40]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Cloud_Cloudy', 'Wind_Light', 'Wind_Moderate', 'Wind_Strong', 'Number_of_ships', 'Ship', 'wetter_sehr_schön_x', 'wetter_sehr_schlecht_x', 'Number_of_ships_scaled', 'Cloud_ok', 'Wind_ok', 'wetter_sehr_schön_y', 'wetter_sehr_schlecht_y']


In [41]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature           VIF
0           Temp_Very_Cold  5.967128e+08
1                Temp_Cold           inf
2                Temp_Mild           inf
3                Temp_Warm           inf
4                 Temp_Hot           inf
5              Cloud_Clear  1.394863e+05
6      Cloud_Partly_Cloudy           inf
7             Cloud_Cloudy           inf
8               Wind_Light           inf
9            Wind_Moderate  1.937751e+05
10             Wind_Strong           inf
11            Weather_Good           inf
12    Weather_Light_Issues           inf
13        Weather_Moderate           inf
14          Weather_Severe           inf
15             KielerWoche  1.167711e+00
16                  Montag           inf
17                Dienstag           inf
18                Mittwoch           inf
19              Donnerstag           inf
20                 Freitag           inf
21                 Samstag           inf
22                 Sonntag           inf
23              

In [42]:
# suche miteinander korrelierrende features


In [43]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Wind_ok', 'Wind_Strong'), ('wetter_sehr_schön_y', 'wetter_sehr_schön_x'), ('Werktag', 'Samstag'), ('Wind_ok', 'Wind_Moderate'), ('Wind_Moderate', 'Wind_Light'), ('wetter_sehr_schön_x', 'Temp_Hot'), ('Cloud_ok', 'Cloud_Clear'), ('Cloud_Cloudy', 'Cloud_Clear'), ('Wind_Strong', 'Wind_Moderate'), ('Cloud_ok', 'Cloud_Cloudy'), ('Sommer', 'Temp_Warm'), ('wetter_sehr_schön_y', 'Temp_Hot'), ('Sommer', 'Temp_Cold'), ('Sommer', 'Umsatz'), ('Winter', 'Temp_Mild'), ('Ship', 'Number_of_ships'), ('Number_of_ships_scaled', 'Ship'), ('Werktag', 'Sonntag'), ('wetter_sehr_schlecht_y', 'wetter_sehr_schlecht_x'), ('Temp_Mild', 'Temp_Cold'), ('Werktag', 'Umsatz'), ('Temp_Cold', 'Umsatz'), ('Heimspiel', 'Samstag'), ('Cloud_ok', 'Cloud_Partly_Cloudy'), ('Cloud_Cloudy', 'Cloud_Partly_Cloudy'), ('Weather_Moderate', 'Weather_Light_Issues'), ('Number_of_ships_scaled', 'Number_of_ships'), ('Weather_Light_Issues', 'Weather_Good'), ('Winter', 'Temp_Cold'), ('is_holiday', 'Umsatz'